# Covid-19 Vaccine Allocations and Case Counts Analysis
----

Written in the Python 3.7.9 Environment

By Tarak Patel, Nicole Lund and Anne Niemiec.

Common Jupyter cells have been notated for ease of merging after individuals their analysis.

Individual working files will be merged using https://pypi.org/project/nbmerge/

In [1]:
### This is a common Jupyter Cell.  Do not modify without coordination. ###

# Load Dependencies

# Data Organization & Collection
import pandas as pd
import requests
import json
from census import Census

# Data Visualization & Analysis
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as st
import gmaps

# General Navigation
import time
import os
import sys

In [2]:
# Define users api key file folder location
api_key_path = r"C:\Users\nlund\Documents\GitHub\untracked_files"

In [3]:
### This is a common Jupyter Cell.  Do not modify without coordination. ###

# Add API key file to system path
sys.path.append(api_key_path)

# Import Census API Key
from untracked_api_keys import census_key
c = Census(census_key, year=2019)

# Import Google API key
from untracked_api_keys import g_key

# Define function for calculating the number of days since the beginning of Vaccine rollout

In [4]:
from datetime import datetime

def get_day_nums(dates,format):
    day_nums = []
    start_date = datetime.strptime('12/14/2020', "%m/%d/%Y")
    for day in dates:
        end_date = datetime.strptime(day, format)
        duration = end_date - start_date
        day_nums.append(duration.days)
    return day_nums

# Load all input data files into DataFrames

In [5]:
# Create File Path for input files. 
csv_pfe =  'source_data/COVID-19_Vaccine_Distribution_Allocations_by_Jurisdiction_-_Pfizer.csv'
csv_mrna = 'source_data/COVID-19_Vaccine_Distribution_Allocations_by_Jurisdiction_-_Moderna.csv'
csv_jnj =  'source_data/COVID-19_Vaccine_Distribution_Allocations_by_Jurisdiction_-_Janssen.csv'
csv_c_d =  'source_data/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv'
state_code = 'source_data/state_code.csv'
csv_administered = 'source_data/us-daily-covid-vaccine-doses-administered.csv'

# Create a dataframe for each file.
pfe_df = pd.read_csv(csv_pfe)
mrna_df = pd.read_csv(csv_mrna)
jnj_df = pd.read_csv(csv_jnj)
c_d_df = pd.read_csv(csv_c_d)
state_code_df = pd.read_csv(state_code)
administered_df = pd.read_csv(csv_administered)

# Prepare state code file for merges

In [7]:
state_code_df = state_code_df.rename(columns={'Postal Code': 'state_code',
                                              'State/District': 'state_name'
                                             })
state_code_df = state_code_df[['state_name','state_code']]
state_code_df.head(3)

,state_name,state_code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ


# Create Vaccine Distribution Allocation DataFrame

Merge Pfizer, Moderna and Johnson & Johnson DataFrames

Merge state postal code

Calculate day_num from the date and add to DataFrame

Reorder columns

In [8]:
# Merge pfe and mrna datframe
pfe_mrna = pd.merge(pfe_df, mrna_df, on=['Jurisdiction', 'Week of Allocations'], how='outer')
pfe_mrna

pfe_mrna_jnj = pd.merge(pfe_mrna, jnj_df, on=['Jurisdiction', 'Week of Allocations'], how='outer')
pfe_mrna_jnj

# Rename Columns in the DataFrame and Fill all the NaN with Zero to work with the data
all_vaccines_df = pfe_mrna_jnj.rename(columns={ 'Jurisdiction': 'state_name',
                                                   'Week of Allocations': 'date',
                                                   '1st Dose Allocations_x': 'pfe_dose_1',
                                                   '2nd Dose Allocations_x': 'pfe_dose_2',
                                                   '1st Dose Allocations_y': 'mrna_dose_1',
                                                   '2nd Dose Allocations_y': 'mrna_dose_2',
                                                   '1st Dose Allocations': 'jnj_dose_1'
                                                   }).fillna(0)


# Correct state_name values
all_vaccines_df.loc[all_vaccines_df['state_name'] == 'Mariana Islands',['state_name']] = 'Northern Mariana Islands'
all_vaccines_df.loc[all_vaccines_df['state_name'] == 'U.S. Virgin Islands',['state_name']] = 'Virgin Islands'

# Merge state_code into the DataFrame
vaccine_w_state_df = pd.merge(all_vaccines_df,state_code_df,on=['state_name'],how='left')

# Correct city values
vaccine_w_state_df.loc[all_vaccines_df['state_name'] == 'Chicago',['state_code']] = 'IL'
vaccine_w_state_df.loc[all_vaccines_df['state_name'] == 'New York City',['state_code']] = 'NY'
vaccine_w_state_df.loc[all_vaccines_df['state_name'] == 'Philadelphia',['state_code']] = 'PA'
vaccine_w_state_df.loc[all_vaccines_df['state_name'] == 'Federal Entities',['state_code']] = 'GV'

# Convert date to day_num after 12/14/2020 and add to DataFrame
day_nums = get_day_nums(vaccine_w_state_df.date.tolist(),"%m/%d/%Y")
vaccine_w_state_df['day_num'] = day_nums

# Reorder columns
Combined_Vaccine_df = vaccine_w_state_df[['state_name','state_code','date','day_num','pfe_dose_1','pfe_dose_2','mrna_dose_1','mrna_dose_2','jnj_dose_1']]

# Export DataFrame to csv
Combined_Vaccine_df.to_csv('clean_data/Combined_Vaccine_df.csv',index=False)

# Display DataFrame
Combined_Vaccine_df.head(3)

,state_name,state_code,date,day_num,pfe_dose_1,pfe_dose_2,mrna_dose_1,mrna_dose_2,jnj_dose_1
0,Connecticut,CT,04/19/2021,126,54990,54990,40400.0,40400.0,0.0
1,Maine,ME,04/19/2021,126,21060,21060,15400.0,15400.0,0.0
2,Massachusetts,MA,04/19/2021,126,105300,105300,77700.0,77700.0,0.0


# Create Administered Vaccine DataFrame

Merge state name

Calculate day_num from the date and add to DataFrame

Reorder columns

In [10]:
# Rename columns
administered_df = administered_df.rename(columns={'Day': 'date',
                                                  'Entity': 'state_name'
                                                 })

# Correct state_name values in state code file
administered_df.loc[administered_df['state_name'] == 'Federated States of Micronesia',['state_name']] = 'Micronesia'
administered_df.loc[administered_df['state_name'] == 'New York State',['state_name']] = 'New York'
administered_df.loc[administered_df['state_name'] == 'Republic of Palau',['state_name']] = 'Palau'

# Merge in state code
administered_w_state_df = pd.merge(administered_df,state_code_df,on=['state_name'],how='left')

# Correct state_name values not in state code file
administered_w_state_df.loc[administered_w_state_df['state_name'] == 'Bureau of Prisons',['state_code']] = 'GV'
administered_w_state_df.loc[administered_w_state_df['state_name'] == 'Dept of Defense',['state_code']] = 'GV'
administered_w_state_df.loc[administered_w_state_df['state_name'] == 'Indian Health Svc',['state_code']] = 'GV'
administered_w_state_df.loc[administered_w_state_df['state_name'] == 'Long Term Care',['state_code']] = 'GV'
administered_w_state_df.loc[administered_w_state_df['state_name'] == 'Veterans Health',['state_code']] = 'GV'

# Convert date to day_num after 12/14/2020 and add to DataFrame
day_nums = get_day_nums(administered_w_state_df.date.tolist(),"%m/%d/%Y")
administered_w_state_df['day_num'] = day_nums

# Reorder columns
Vaccines_Administered_df = administered_w_state_df[['state_name','state_code','date','day_num','daily_vaccinations']]

# Export DataFrame to csv
Vaccines_Administered_df.to_csv('clean_data/Vaccines_Administered_df.csv',index=False)

# Display DataFrame
Vaccines_Administered_df.head(3)

,state_name,state_code,date,day_num,daily_vaccinations
0,Alabama,AL,1/13/2021,30,5906
1,Alabama,AL,1/14/2021,31,7083
2,Alabama,AL,1/15/2021,32,7478


# Create Covid Cases & Death DataFrame

Merge state name

Calculate day_num from the date and add to DataFrame

Reorder columns

Remove data prior to Vaccination rollout on 12/14/2020

In [11]:
#Rename Columns in the DataFrame and Fill all the NaN with Zero to work with the data
c_d_df_rename = c_d_df.rename(columns={'submission_date': 'date',
                                             'state': 'state_code',
                                             'tot_cases': 'total_cases',
                                             'conf_cases': 'confirmed_cases',
                                             'prob_cases': 'probable_cases',
                                             'new_case': 'new_cases',
                                             'pnew_case': 'pnew_cases',
                                             'tot_death': 'total_deaths',
                                             'conf_death': 'confirmed_deaths',
                                             'prob_death': 'probable_deaths',
                                             'new_death': 'new_deaths',
                                             'pnew_death': 'pnew_deaths'
                                             }).fillna(0)

# Correct state_name and state_code for corner cases
c_d_df_rename.loc[c_d_df_rename['state_code'] == 'FSM',['state_code']] = 'FM'
c_d_df_rename.loc[c_d_df_rename['state_code'] == 'RMI',['state_code']] = 'MH'

# Merge in state name
c_d_w_state_df = pd.merge(c_d_df_rename,state_code_df,on=['state_code'],how='left')

# Correct NYC state name and code
c_d_w_state_df.loc[c_d_w_state_df['state_code'] == 'NYC',['state_name']] = 'New York City'
c_d_w_state_df.loc[c_d_w_state_df['state_code'] == 'NYC',['state_code']] = 'NY'

# Convert date to day_num after 12/14/2020 and add to DataFrame
day_nums = get_day_nums(c_d_w_state_df.date.tolist(),"%m/%d/%Y")
c_d_w_state_df['day_num'] = day_nums

# Reorder columns and remove data prior to 12/14/2020
c_d_withstate_df = c_d_w_state_df.loc[c_d_w_state_df['day_num'] >= 0,['state_name','state_code','date','day_num','total_cases','confirmed_cases','probable_cases','new_cases','pnew_cases','total_deaths','confirmed_deaths','probable_deaths','new_deaths','pnew_deaths','consent_cases','consent_deaths','created_at']]

# Export DataFrame to csv
c_d_withstate_df.to_csv('clean_data/c_d_withstate_df.csv',index=False)

# Display DataFrame
c_d_withstate_df.head(3)

,state_name,state_code,date,day_num,total_cases,confirmed_cases,probable_cases,new_cases,pnew_cases,total_deaths,confirmed_deaths,probable_deaths,new_deaths,pnew_deaths,consent_cases,consent_deaths,created_at
19620,Alaska,AK,12/14/2020,0,40160,0.0,0.0,422,0.0,175,0.0,0.0,0,0.0,0,0,12/15/2020 14:39
19621,Alabama,AL,12/14/2020,0,297895,244762.0,53133.0,2264,332.0,4102,3624.0,478.0,0,0.0,Agree,Agree,12/15/2020 14:39
19622,Arkansas,AR,12/14/2020,0,187057,0.0,0.0,1355,122.0,2990,0.0,0.0,45,22.0,Not agree,Not agree,12/15/2020 14:39


# Collect Census Population Data

In [12]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E", "B23025_005E"), {'for': 'state:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={
                                      "B01003_001E": "population",
                                      "B01002_001E": "median_age",
                                      "B19013_001E": "household_income",
                                      "B19301_001E": "per_capita_income",
                                      "B17001_002E": "poverty_count",
                                      "B23025_005E": "unemployment_count",
                                      "NAME": "state_name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["poverty_rate"] = 100 * \
    census_pd["poverty_count"].astype(
        int) / census_pd["population"].astype(int)

census_pd["unemployment_rate"] = 100 * \
    census_pd["unemployment_count"].astype(
        int) / census_pd["population"].astype(int)


# Merge in state name
census_w_state_pd = pd.merge(census_pd,state_code_df,on=['state_name'],how='left')


# Reorder columns
census_w_state_pd = census_w_state_pd[["state_name","state_code","population", "median_age", "household_income",
                       "per_capita_income", "poverty_count", "poverty_rate", "unemployment_count", "unemployment_rate"]]

# Export DataFrame to csv
census_w_state_pd.to_csv('clean_data/Population.csv',index=False)

# Display DataFrame
census_w_state_pd.head(3)

,state_name,state_code,population,median_age,household_income,per_capita_income,poverty_count,poverty_rate,unemployment_count,unemployment_rate
0,Alabama,AL,4876250.0,39.0,50536.0,27928.0,795989.0,16.323794,132095.0,2.708946
1,Alaska,AK,737068.0,34.3,77640.0,36787.0,76933.0,10.437707,26808.0,3.637114
2,Arizona,AZ,7050299.0,37.7,58945.0,30694.0,1043764.0,14.804535,195905.0,2.778676
